In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score
import glob
import os
import tensorflow as tf
%matplotlib inline

## Model

In [2]:
def conf_mat(pred,truth):
    print( np.count_nonzero(pred == truth) )
    print( len(pred) )
    res = [0,0,0,0]
    a = 0
    
    for i in range(len(truth)):
        if truth[i] == 1:
            if truth[i] == pred[i]:
                a = 0
            else:
                a = 2
                
        else:
            if truth[i] == pred[i]:
                a = 1
            else:
                a = 3
                
        res[a] = res[a] + 1
     
    print(res)
    return res

## Multi-dataset Model

In [5]:
input_dir = './../train/KPI/'

summary = pd.DataFrame(columns=['KPI', 'TP', 'TN', 'FP', 'FN', 'PRECISION', 'RECALL', 'F1_SCORE'])

for fname in os.listdir(input_dir):
    df  = pd.read_csv(os.path.join(input_dir, fname), index_col='timestamp')
    kpi_name = df['KPI ID'].values[0]
    print(kpi_name)
    df = df.drop(['KPI ID'], axis=1)
    
    # Normalize Values
    normalized_df=(df-df.min())/(df.max()-df.min())
    normalized_df = normalized_df.astype({'label': 'int64'})
    
    # Split to Train and Test
    train_set, test_set= np.split(normalized_df, [int(.75 *len(normalized_df))])
    
    # Format Train and Test
    X = np.array(train_set['value']).reshape(-1, 1)
    y = np.array(train_set['label'])
    x_test = np.array(test_set['value']).reshape(-1,1)
    y_test = np.array(test_set['label'])
    
    
    # Check Valid Train Dataset
    if len(np.unique(y)) > 1:

        # Train Model
        model = KMeans(n_clusters=2).fit(X)
#         print(y_pred)

        # Make Predictions
        predictions = model.predict(x_test)
        print(np.unique(predictions))
#         print(shit)

        # Compute Confusion Matrix
        cf = conf_mat(predictions,y_test) 

        # F1-Score
        prec = 0
        rec = 0
        f1 = 0
        if (cf[0] + cf[2]) != 0:
            prec = cf[0] / (cf[0] + cf[2])
        if (cf[0] + cf[3]) != 0:
            rec  = cf[0] / (cf[0] + cf[3])
        if (prec + rec) != 0:
            f1   = 2 * (prec * rec / (prec+rec))

    #     print(f1_score(predictions,y_test))


        summary = summary.append({'KPI': kpi_name, 
                                   'TP': cf[0],  
                                   'TN': cf[1], 
                                   'FP': cf[2], 
                                   'FN': cf[3], 
                                   'PRECISION': prec, 
                                   'RECALL': rec,     
                                   'F1_SCORE': f1 }, ignore_index=True)
        
    else:
        summary = summary.append({'KPI': kpi_name, 
                                   'TP': None,  
                                   'TN': None, 
                                   'FP': None, 
                                   'FN': None, 
                                   'PRECISION': None, 
                                   'RECALL': None,     
                                   'F1_SCORE': None }, ignore_index=True)
    

a40b1df87e3f1c87
[0 1]
11116
34482
[58, 11058, 48, 23318]
b3b2e6d1a791d63a
[0 1]
1620
2062
[0, 1620, 0, 442]
da403e4e3f87c9e0
[0 1]
15619
32259
[25, 15594, 237, 16403]
046ec29ddf80d62e
[0 1]
1257
2196
[0, 1257, 0, 939]
18fbb1d5a5dc099d
[0]
32231
32282
[0, 32231, 51, 0]
09513ae3e75778a3
[0 1]
17416
32243
[118, 17298, 10, 14817]
07927a9a18fa19ae
[0 1]
1747
2740
[15, 1732, 9, 984]
cff6d3c01e6a6bfa
[0 1]
21772
36908
[254, 21518, 35, 15101]
54e8a140f6237526
[0 1]
737
2062
[0, 737, 0, 1325]
c58bfcbacb2822d1
[0 1]
8446
32167
[36, 8410, 11, 23710]
7c189dd36f048a6c
[0 1]
11571
36923
[46, 11525, 34, 25318]
8bef9af9a922e0b3
[0 1]
23088
32364
[22, 23066, 54, 9222]
40e25005ff8992bd
[0 1]
7516
25064
[75, 7441, 85, 17463]
76f4550c43334374
[0 1]
971
2196
[2, 969, 2, 1223]
affb01ca2b4f0b45
[0 1]
26609
36920
[36, 26573, 57, 10254]
88cf3a776ba00e7c
[0 1]
12401
16359
[636, 11765, 175, 3783]
9bd90500bfd11edb
[0 1]
30616
32154
[0, 30616, 0, 1538]
e0770391decc44ce
[0 1]
25751
36756
[2583, 23168, 123, 10882]


In [6]:
summary.to_csv('KMeans.csv')